### 최초 청크 : 최초 1번만 실행

In [ ]:
from google.colab import drive
! pip install pyLDAvis

drive.mount('/content/drive')

# 최초 1회 돌리면 오류가 뜰겁니다
# 에러 창에서 Restart Runtime 버튼 누르세요
# 버튼 누르고 난 후에 다시 실행하면 됩니다.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 데이터 불러오기
import pandas as pd
import numpy as np

## 이 청크를 전체 복사해서 맡은 DF 개수만큼 붙여넣기 해서 각자 돌리시면 됩니다
#### 이 청크 위 최초 청크는 처음 한 번만 돌리면 됨
#### 경제&사회 10월 18일

In [133]:
# 데이터 잘 가져오기
data = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/기사크롤링_TFIDF/경제&사회_20211018_TFIDF.csv', encoding = 'utf-8')

In [134]:
# 모델 돌리는 거 자체에 필요한 코드는 아니지만 저장하면서 index list unlist 해주는 게 편하니까 그냥 돌려주세요

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features = 1000)
x = tfidf_vectorizer.fit(data.clean_as_sentence)
tfidf_df = pd.DataFrame(x.transform(data.clean_as_sentence).toarray(), columns = sorted(tfidf_vectorizer.vocabulary_))

important_words = []
for i in range(0, len(tfidf_df)):
  a = []
  imp_word = tfidf_df.iloc[i, :].index[tfidf_df.iloc[i, :].values > 0.05].tolist()
  a.append(imp_word)
  important_words.append(a)

data['important_words'] = important_words

In [135]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#LDA 는 Count기반의 Vectorizer만 적용 
count_vectorizer = CountVectorizer(max_df=0.1, max_features=1000, min_df=1, ngram_range=(1,2))

feat_vect = count_vectorizer.fit_transform(data.clean_words)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (2761, 1000)


In [136]:
# 모델 파라미터 CV

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

search_params = {'n_components': [3, 5, 7, 9, 11], 
                 'learning_decay': [.7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
# 10분 이상 소요될 수 있음
model.fit(feat_vect)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:440: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:93: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more d

GridSearchCV(cv=None, error_score=nan,
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                                 tota

In [137]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(feat_vect))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 11}
Best Log Likelihood Score:  -272805.0413187047
Model Perplexity:  404.120599323083


In [138]:
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [139]:
import pyLDAvis.sklearn  # sklearn의 ldamodel에 최적화된 라이브러리

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(best_lda_model, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [140]:
def show_topics(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=count_vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(1, df_topic_keywords.shape[0]+1)]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 1,채권,대회,현대차,인수,선수,호텔,gs,영상,esg,현대,수소,우승,암모니아,wrc,랠리
Topic 2,주식,포인트,오른,회장,담보,외국인,lg,매도,수출,종목,매수,물류,임시,cj,투입
Topic 3,이익,백신,접종,보험,배당,영업 이익,주식,수익,감소,sk,회복,급등,유가,백신 접종,동기
Topic 4,전기차,배터리,솔루션,lg,에너지 솔루션,합작,법인,공장,스텔 란티스,란티스,스텔,lg 에너지,합작 법인,설립,북미
Topic 5,콘텐츠,게임,버스,티빙,메타,메타 버스,넷플릭스,네이버,ott,제작,nft,드라마,규제,페이스북,디즈니
Topic 6,반도체,직원,채용,교육,수수료,바이오,기능,인력,카드,프로그램,전국,공급망,대회,아이디어,전력
Topic 7,보험,의원,탄소,공사,탄소 중립,중립,예금,사장,국회,예금 보험,감축,사기,정무,적발,상향
Topic 8,비트코인,etf,kb,뱅크,카카오,인증,카카오 뱅크,주행,코인,선물,모바일,한화,화폐,운용,전자서명
Topic 9,대출,주택,전세,가계,신용,규제,nh,한도,부동산,전세 대출,당국,모니터,가계 대출,ai,시중
Topic 10,친환경,브랜드,상장,중공업,수주,ai,청약,연구,발행,소재,시몬,치료,공모,환자,유럽


In [141]:
# 문서별 토픽 할당

doc_topic = best_lda_model.transform(feat_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
  topic_most_pr = doc_topic[n].argmax() +1
  topic_pr = doc_topic[n].max()
  doc_per_topic_list.append([n, topic_most_pr, topic_pr])

doc_topic_df = pd.DataFrame(doc_per_topic_list, columns = ['Doc_Num', 'Topic', 'Percentage'])

In [142]:
data['topic'] = doc_topic_df['Topic']
data['Percentage'] = doc_topic_df['Percentage']

In [143]:
# 저장하는 파일명 잘 확인

df_topic_keywords.to_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/토픽모델링/경제&사회_20211018_Topics&Keywords.csv', index = False, encoding = 'utf-8')
data.to_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/토픽모델링/경제&사회_20211018_Topic_Allocation.csv', index = False, encoding = 'utf-8')

## 이 청크를 전체 복사해서 맡은 DF 개수만큼 붙여넣기 해서 각자 돌리시면 됩니다
#### 이 청크 위 최초 청크는 처음 한 번만 돌리면 됨
#### 경제&사회 10월 19일

In [ ]:
# 데이터 잘 가져오기
data = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/기사크롤링_TFIDF/경제&사회_20211019_TFIDF.csv', encoding = 'utf-8')

In [ ]:
# 모델 돌리는 거 자체에 필요한 코드는 아니지만 저장하면서 index list unlist 해주는 게 편하니까 그냥 돌려주세요

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features = 1000)
x = tfidf_vectorizer.fit(data.clean_as_sentence)
tfidf_df = pd.DataFrame(x.transform(data.clean_as_sentence).toarray(), columns = sorted(tfidf_vectorizer.vocabulary_))

important_words = []
for i in range(0, len(tfidf_df)):
  a = []
  imp_word = tfidf_df.iloc[i, :].index[tfidf_df.iloc[i, :].values > 0.05].tolist()
  a.append(imp_word)
  important_words.append(a)

data['important_words'] = important_words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#LDA 는 Count기반의 Vectorizer만 적용 
count_vectorizer = CountVectorizer(max_df=0.1, max_features=1000, min_df=1, ngram_range=(1,2))

feat_vect = count_vectorizer.fit_transform(data.clean_words)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (2977, 1000)


In [ ]:
# 모델 파라미터 CV

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

search_params = {'n_components': [3, 5, 7, 9, 11], 
                 'learning_decay': [.7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
# 10분 이상 소요될 수 있음
model.fit(feat_vect)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:440: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:93: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more d

GridSearchCV(cv=None, error_score=nan,
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                                 tota

In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(feat_vect))

Best Model's Params:  {'learning_decay': 0.7, 'n_components': 11}
Best Log Likelihood Score:  -286862.87252695736
Model Perplexity:  416.2115485176301


In [ ]:
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pyLDAvis.sklearn  # sklearn의 ldamodel에 최적화된 라이브러리

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(best_lda_model, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
def show_topics(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=count_vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(1, df_topic_keywords.shape[0]+1)]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 1,현대차,유럽,제도,토스,기아,중소기업,독일,영국,현대차 기아,임금,점유,전기차,조사,자금,자동차
Topic 2,보험,반도체,소재,sk,열분해,플라스틱,화학,애플,lg,중고차,손해,공장,에코,패널,트릭
Topic 3,바이오,수소,치료,영상,제보,수출,면역,물류,신약,운송,또는,중소기업,hmm,프로,글로비스
Topic 4,삼성,카드,배터리,전기차,계약,lg,솔루션,란티스,sdi,삼성 sdi,스텔,스텔 란티스,공장,에너지,수수료
Topic 5,사모펀드,운용,전시회,한화,esg,무인기,전시,선박,드론,위성,펀드,경영,항공우주,통상,adex
Topic 6,대출,비트코인,etf,주택,신용,담보,신용 대출,담보 대출,선물,주택 담보,화폐,받은,의원,자금,전세
Topic 7,백신,에너지,접종,감소,규제,발행,우려,정책,탄소,수소,국가,경기,회장,감축,도시
Topic 8,직원,게임,자산운용,버스,거래소,메타,이벤트,채용,메타 버스,교육,체인,블록,콘텐츠,광주,회원
Topic 9,ai,브랜드,모델,연구,lg,스마트,데이터,솔루션,디자인,지능,세대,혁신,테크,기능,소비자
Topic 10,오른,종목,매수,넷플릭스,탄소,외국인,게임,강세,테슬라,대감,콘텐츠,카카오,개인,전일,상위


In [ ]:
# 문서별 토픽 할당

doc_topic = best_lda_model.transform(feat_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
  topic_most_pr = doc_topic[n].argmax() +1
  topic_pr = doc_topic[n].max()
  doc_per_topic_list.append([n, topic_most_pr, topic_pr])

doc_topic_df = pd.DataFrame(doc_per_topic_list, columns = ['Doc_Num', 'Topic', 'Percentage'])

In [ ]:
data['topic'] = doc_topic_df['Topic']
data['Percentage'] = doc_topic_df['Percentage']

In [ ]:
# 저장하는 파일명 잘 확인

df_topic_keywords.to_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/토픽모델링/경제&사회_20211019_Topics&Keywords.csv', index = False, encoding = 'utf-8')
data.to_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/토픽모델링/경제&사회_20211019_Topic_Allocation.csv', index = False, encoding = 'utf-8')

## 이 청크를 전체 복사해서 맡은 DF 개수만큼 붙여넣기 해서 각자 돌리시면 됩니다
#### 이 청크 위 최초 청크는 처음 한 번만 돌리면 됨
#### 경제&사회 10월 20일

In [ ]:
# 데이터 잘 가져오기
data = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/기사크롤링_TFIDF/경제&사회_20211020_TFIDF.csv', encoding = 'utf-8')

In [ ]:
# 모델 돌리는 거 자체에 필요한 코드는 아니지만 저장하면서 index list unlist 해주는 게 편하니까 그냥 돌려주세요

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features = 1000)
x = tfidf_vectorizer.fit(data.clean_as_sentence)
tfidf_df = pd.DataFrame(x.transform(data.clean_as_sentence).toarray(), columns = sorted(tfidf_vectorizer.vocabulary_))

important_words = []
for i in range(0, len(tfidf_df)):
  a = []
  imp_word = tfidf_df.iloc[i, :].index[tfidf_df.iloc[i, :].values > 0.05].tolist()
  a.append(imp_word)
  important_words.append(a)

data['important_words'] = important_words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#LDA 는 Count기반의 Vectorizer만 적용 
count_vectorizer = CountVectorizer(max_df=0.1, max_features=1000, min_df=1, ngram_range=(1,2))

feat_vect = count_vectorizer.fit_transform(data.clean_words)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (3043, 1000)


In [ ]:
# 모델 파라미터 CV

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

search_params = {'n_components': [3, 5, 7, 9, 11], 
                 'learning_decay': [.7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
# 10분 이상 소요될 수 있음
model.fit(feat_vect)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:440: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:93: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more d

GridSearchCV(cv=None, error_score=nan,
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                                 tota

In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(feat_vect))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 11}
Best Log Likelihood Score:  -288629.02176131215
Model Perplexity:  404.8322614581362


In [ ]:
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pyLDAvis.sklearn  # sklearn의 ldamodel에 최적화된 라이브러리

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(best_lda_model, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
def show_topics(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=count_vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(1, df_topic_keywords.shape[0]+1)]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 1,백신,접종,롯데,행사,여행,항공,할인,노선,백신 접종,가입,모바일,대한항공,인천,이벤트,운항
Topic 2,갤럭시,브랜드,디자인,에디,콘텐츠,키츠네,소비자,선박,색상,시스템,메종,메종 키츠네,모델,스포크,라이브
Topic 3,배터리,lg,솔루션,에너지,치료,바이오,에너지 솔루션,lg 에너지,공장,체결,협약,전기차,ai,협력,소프트웨어
Topic 4,게임,넷플릭스,esg,영상,제보,오징어,오징어 게임,선수,가입자,독자,화제,언론,또는,경영,공감
Topic 5,반도체,sk,건설,탄소,장비,hbm,중립,탄소 중립,sk 하이닉스,하이닉스,감축,기계,메모리,공장,에너지
Topic 6,중소기업,소재,혁신,미래,세대,협력,개최,분야,변화,첨단,gm,국제,전시회,테크,기업은행
Topic 7,카카오,이익,페이,인상,카카오 페이,종목,매수,주식,공모,외국인,영업 이익,수수료,청약,개인,비중
Topic 8,비트코인,etf,화폐,거래소,선물,lg,임원,가상,게임,펀드,오른,비트코인 etf,암호,오락실,뉴욕
Topic 9,주택,dsr,신용,금융 당국,뱅크,가계,가계부채,중단,가계 대출,카카오 뱅크,신용 대출,카카오,kb,청년,전세
Topic 10,보험,인수,모터,쌍용차,에디슨,에디슨 모터,채권,선정,발행,손해,생명,협상,매각,법원,자동차


In [ ]:
# 문서별 토픽 할당

doc_topic = best_lda_model.transform(feat_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
  topic_most_pr = doc_topic[n].argmax() +1
  topic_pr = doc_topic[n].max()
  doc_per_topic_list.append([n, topic_most_pr, topic_pr])

doc_topic_df = pd.DataFrame(doc_per_topic_list, columns = ['Doc_Num', 'Topic', 'Percentage'])

In [ ]:
data['topic'] = doc_topic_df['Topic']
data['Percentage'] = doc_topic_df['Percentage']

In [ ]:
# 저장하는 파일명 잘 확인

df_topic_keywords.to_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/토픽모델링/경제&사회_20211020_Topics&Keywords.csv', index = False, encoding = 'utf-8')
data.to_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/토픽모델링/경제&사회_20211020_Topic_Allocation.csv', index = False, encoding = 'utf-8')

## 이 청크를 전체 복사해서 맡은 DF 개수만큼 붙여넣기 해서 각자 돌리시면 됩니다
#### 이 청크 위 최초 청크는 처음 한 번만 돌리면 됨
#### 경제&사회 10월 21일

In [ ]:
# 데이터 잘 가져오기
data = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/기사크롤링_TFIDF/경제&사회_20211021_TFIDF.csv', encoding = 'utf-8')

In [ ]:
# 모델 돌리는 거 자체에 필요한 코드는 아니지만 저장하면서 index list unlist 해주는 게 편하니까 그냥 돌려주세요

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features = 1000)
x = tfidf_vectorizer.fit(data.clean_as_sentence)
tfidf_df = pd.DataFrame(x.transform(data.clean_as_sentence).toarray(), columns = sorted(tfidf_vectorizer.vocabulary_))

important_words = []
for i in range(0, len(tfidf_df)):
  a = []
  imp_word = tfidf_df.iloc[i, :].index[tfidf_df.iloc[i, :].values > 0.05].tolist()
  a.append(imp_word)
  important_words.append(a)

data['important_words'] = important_words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#LDA 는 Count기반의 Vectorizer만 적용 
count_vectorizer = CountVectorizer(max_df=0.1, max_features=1000, min_df=1, ngram_range=(1,2))

feat_vect = count_vectorizer.fit_transform(data.clean_words)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (3419, 1000)


In [ ]:
# 모델 파라미터 CV

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

search_params = {'n_components': [3, 5, 7, 9, 11], 
                 'learning_decay': [.7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
# 10분 이상 소요될 수 있음
model.fit(feat_vect)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:440: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:93: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more d

GridSearchCV(cv=None, error_score=nan,
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                                 tota

In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(feat_vect))

Best Model's Params:  {'learning_decay': 0.7, 'n_components': 11}
Best Log Likelihood Score:  -330063.48303091194
Model Perplexity:  374.47996170280453


In [ ]:
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pyLDAvis.sklearn  # sklearn의 ldamodel에 최적화된 라이브러리

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(best_lda_model, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
def show_topics(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=count_vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(1, df_topic_keywords.shape[0]+1)]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 1,발사,누리,발사체,우주,위성,엔진,제작,우주 센터,한화,전남,나로,나로 우주,발사체 누리,우리금융,시험
Topic 2,제보,lng,용접,소중한,언론,화제,공감,있다고 판단,하겠습니다,상부,받습니다,적극 반영,기다립니다 가치나,영상 제보,제보 기다립니다
Topic 3,테슬라,보험,반도체,인수,포인트,영업 이익,모터,오른,매수,감소,평균,에디슨,우려,에디슨 모터,유가
Topic 4,sk,바이오,삼성,직원,경영,esg,치료,kt,인증,혁신,업무,데이터,배달,교육,갤럭시
Topic 5,비트코인,etf,화폐,펀드,가상,비트코인 etf,암호,거래소,코인,암호 화폐,선물,비트코인 가격,가상 화폐,수익률,비트코인 선물
Topic 6,국정감사,지적,정무,질의,정무 위원회,수수료,협약,민주당,국회 열린,감사,위원장,원장,금감원,업무,여의도
Topic 7,물류,etn,계약,두산,행사,통상,포스코,원전,케미칼,중공업,무역,cj,발전,두산 중공업,체결
Topic 8,브랜드,디자인,lg,배터리,소비자,미래,롯데,수소,탄소,전기차,브랜드 가치,맥주,공모,모빌리티,자동차
Topic 9,kb,청년,lg,kb 금융,백신,일자리,순이익,회장,수수료,채용,접종,이자,누적,배당,희망
Topic 10,대출,규제,상환,dsr,가계부채,주택,자금,가계,당국,가계 대출,발행,대책,부동산,채권,전세


In [ ]:
# 문서별 토픽 할당

doc_topic = best_lda_model.transform(feat_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
  topic_most_pr = doc_topic[n].argmax() +1
  topic_pr = doc_topic[n].max()
  doc_per_topic_list.append([n, topic_most_pr, topic_pr])

doc_topic_df = pd.DataFrame(doc_per_topic_list, columns = ['Doc_Num', 'Topic', 'Percentage'])

In [ ]:
data['topic'] = doc_topic_df['Topic']
data['Percentage'] = doc_topic_df['Percentage']

In [ ]:
# 저장하는 파일명 잘 확인

df_topic_keywords.to_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/토픽모델링/경제&사회_20211021_Topics&Keywords.csv', index = False, encoding = 'utf-8')
data.to_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/토픽모델링/경제&사회_20211021_Topic_Allocation.csv', index = False, encoding = 'utf-8')

## 이 청크를 전체 복사해서 맡은 DF 개수만큼 붙여넣기 해서 각자 돌리시면 됩니다
#### 이 청크 위 최초 청크는 처음 한 번만 돌리면 됨
#### 경제&사회 10월 22일

In [122]:
# 데이터 잘 가져오기
data = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/기사크롤링_TFIDF/경제&사회_20211022_TFIDF.csv', encoding = 'utf-8')

In [123]:
# 모델 돌리는 거 자체에 필요한 코드는 아니지만 저장하면서 index list unlist 해주는 게 편하니까 그냥 돌려주세요

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features = 1000)
x = tfidf_vectorizer.fit(data.clean_as_sentence)
tfidf_df = pd.DataFrame(x.transform(data.clean_as_sentence).toarray(), columns = sorted(tfidf_vectorizer.vocabulary_))

important_words = []
for i in range(0, len(tfidf_df)):
  a = []
  imp_word = tfidf_df.iloc[i, :].index[tfidf_df.iloc[i, :].values > 0.05].tolist()
  a.append(imp_word)
  important_words.append(a)

data['important_words'] = important_words

In [124]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#LDA 는 Count기반의 Vectorizer만 적용 
count_vectorizer = CountVectorizer(max_df=0.1, max_features=1000, min_df=1, ngram_range=(1,2))

feat_vect = count_vectorizer.fit_transform(data.clean_words)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (2101, 1000)


In [125]:
# 모델 파라미터 CV

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

search_params = {'n_components': [3, 5, 7, 9, 11], 
                 'learning_decay': [.7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
# 10분 이상 소요될 수 있음
model.fit(feat_vect)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:440: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:93: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more d

GridSearchCV(cv=None, error_score=nan,
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                                 tota

In [126]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(feat_vect))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 11}
Best Log Likelihood Score:  -205302.22382301104
Model Perplexity:  401.75147498855245


In [127]:
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [128]:
import pyLDAvis.sklearn  # sklearn의 ldamodel에 최적화된 라이브러리

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(best_lda_model, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [129]:
def show_topics(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=count_vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(1, df_topic_keywords.shape[0]+1)]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 1,발행,계약,누리,발사,위성,운용,감소,우주,펀드,dls,투자 증권,금액,한화,회장,교보
Topic 2,백신,게임,브랜드,시설,치료,바이오,접종,호텔,모바일,센터,디자인,확인,온라인,품질,방식
Topic 3,순이익,이자,동기,누적,동기 대비,대비 증가,채권,수수료,헝다,배당,수익,kb,비율,영업 이익,누적 순이익
Topic 4,카드,esg,규제,경영,당국,청년,수출,롯데,전세,가계,제주,친환경,신용,조치,비중
Topic 5,lg,ai,영상,네이버,협약,kt,물류,스마트,제보,일자리,업무 협약,스타트업,청년,채용,촬영
Topic 6,반도체,에너지,오른,종목,외국인,테슬라,바이오,가스,강세,매수,전일,유가,애플,반면,개인
Topic 7,모터,에디슨,에디슨 모터,무인기,인수,쌍용차,회장,위원장,대한항공,협력,씨티,조달,노조,씨티 은행,매각
Topic 8,비트코인,보험,화폐,가상,버스,etf,메타,거래소,메타 버스,암호,암호 화폐,대통령,포스코,코인,트럼프
Topic 9,배터리,탄소,전기차,삼성,법인,중립,탄소 중립,합작,sdi,삼성 sdi,합작 법인,란티스,스텔 란티스,스텔,공장
Topic 10,부원장,금감원,교육,인사,빅데이터,인재,부원,장보,부원 장보,국장,etn,나달,기아,임명,원장


In [130]:
# 문서별 토픽 할당

doc_topic = best_lda_model.transform(feat_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
  topic_most_pr = doc_topic[n].argmax() +1
  topic_pr = doc_topic[n].max()
  doc_per_topic_list.append([n, topic_most_pr, topic_pr])

doc_topic_df = pd.DataFrame(doc_per_topic_list, columns = ['Doc_Num', 'Topic', 'Percentage'])

In [131]:
data['topic'] = doc_topic_df['Topic']
data['Percentage'] = doc_topic_df['Percentage']

In [132]:
# 저장하는 파일명 잘 확인

df_topic_keywords.to_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/토픽모델링/경제&사회_20211022_Topics&Keywords.csv', index = False, encoding = 'utf-8')
data.to_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/토픽모델링/경제&사회_20211022_Topic_Allocation.csv', index = False, encoding = 'utf-8')